<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/%ED%95%9C%EA%B8%80%EB%AC%B8%EC%9E%A5%EC%88%98%EC%A7%91_1027.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한글 뉴스 크롤러

In [29]:
import sys
import requests
import pandas as pd
import datetime
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from multiprocessing import Pool

client_id = "waXxTjNdGjzj8xtT3nbF" #1.에서 취득한 아이디 넣기
client_secret = "H6mejnkuYx"  #1. 에서 취득한 키 넣기


# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    txt = txt.replace('”','')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def clean_html(x):
    x = re.sub("\&\w*\;","",x)
    x = re.sub("<.*?>","",x)
    return x

def get_news_urls(search_word = '코로나',max_display = 100):
    encode_type = 'json' #출력 방식 json 또는 xml
    #max_display = 100 #출력 뉴스 수
    sort = 'date' #결과값의 정렬기준 시간순 date, 관련도 순 sim
    start = 1 # 출력 위치

    url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

    #print(url)
    #헤더에 아이디와 키 정보 넣기
    headers = {'X-Naver-Client-Id' : client_id,
            'X-Naver-Client-Secret':client_secret
            }

    df = None
    #HTTP요청 보내기
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        #요청 결과 보기 200 이면 정상적으로 요청 완료
        df = pd.DataFrame(r.json()['items'])
        df['title'] = df['title'].apply(lambda x: clean_html(x))
        df['description'] = df['description'].apply(lambda x: clean_html(x))
    return df

def get_news_title_contents(url):
    headers = {'User-Agent':'Chrome/66.0.3359.181'}
    main_news = []
    result = requests.get(url,headers=headers)
    if result.status_code == 200:
        bs_obj = BeautifulSoup(result.content, "html.parser")
        #print(bs_obj)
        try:
            title = bs_obj.find("h2",{"class":"end_tit"}).text
            body = bs_obj.find("div",{"class":"article_body"}).get_text("\n").replace("// flash 오류를 우회하기 위한 함수 추가","").replace("function _flash_removeCallback() {}","")
            #aid = bs_obj.find("div",{"class":"article_header"}).find("img")["title"]
            main_news = [clean_text(title),clean_text(body)]
        except:
            pass
            #print("Unexpected error:", sys.exc_info()[0])
    return main_news    

In [30]:
def make_corpus(search_word = '코로나'):
    f = open("korean_news_corpus.csv", "a")
    df = get_news_urls(search_word = search_word,max_display = 100)
    if df is not None:
        for url in df['link']:
            if url.startswith('https://n.news.naver.com/mnews/'):
                
                news = get_news_title_contents(url)
                print(news)
                if (len(news) == 2):
                    print('\rtitle:',news[0],end="", flush=True)
                    f.write(news[0] + "," + news[1] + "\r\n")
    f.close() 

In [ ]:
import random

# 여기에다가 필요한 분야의 키워드를 나열한다!

source = """
트와이스 안다라수마나라 안유진
"""
source = clean_text(source)
source = source.split(' ')


while(True):
    keyword = random.choice(source)
    try:
        make_corpus(search_word = keyword)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        print("Unexpected error:", sys.exc_info()[0])     


In [36]:
df = pd.read_csv('korean_news_corpus.csv')
df.columns = ['title','contents']
df

,title,contents
0,트와이스 정연 스타랭킹 女 아이돌 3주 연속 정상,/사진스타랭킹 그룹 트와이스 정연이 스타랭킹 여자 아이돌 부문 1위를 차지했다. 정...
1,트와이스 지효 동생 화보 공개 “제니랑 똑같네,최근 패션 브랜드 화보를 찍은 모델 박지영. 럭스 엘에프 인스타그램 캡처. 그룹 트...
2,강남부자 강종현 결별 박민영 2022 AAA 출격한다,[텐아시아류예지 기자] 독보적인 비주얼과 연기력으로 사랑받는 배우들이 ‘2022 A...
3,아이브 장원영 얼굴 천재·무대 천재·MC 천재,스타쉽엔터테인먼트 제공 [OSEN장우영 기자] MZ 워너비 아이콘 아이브(IVE :...
4,안유진 하트 뿅뿅 부르는 가을 여신美,아이브 안유진. /사진안유진 인스타그램 그룹 아이브(IVE) 멤버 안유진이 근황을 ...
5,아이브 장원영 2년 연속 ‘AAA’ MC 확정,아이브 장원영 프로필. 스타쉽엔터테인먼트 제공 걸그룹 아이브(IVE : 안유진 가을...
6,아이브 안유진 흰티+청바지…심쿵 유발하는 여신미,그룹 아이브(IVE) 안유진이 청순한 비주얼을 자랑했다. 사진안유진 인스타그램 지난...
7,대세 아이브 장원영… 2년 연속 2022 AAA MC 맡는다,아이브(IVE : 안유진 가을 레이 장원영 리즈 이서)의 멤버 장원영이 2년 연속으...
8,장원영 2년 연속 AAA MC 발탁…다방면 활약,아이브 장원영ⓒ 뉴스1 (서울뉴스1) 황미현 기자 아이브(IVE : 안유진 가을 레...
9,MZ 아이콘 아이브 장원영 2년 연속 AAA MC 확정활약 기대[공식],/사진제공스타쉽엔터테인먼트 MZ 워너비 아이콘 아이브(IVE 안유진 가을 레이 장원...


In [37]:
df['contents']

0     /사진스타랭킹 그룹 트와이스 정연이 스타랭킹 여자 아이돌 부문 1위를 차지했다. 정...
1     최근 패션 브랜드 화보를 찍은 모델 박지영. 럭스 엘에프 인스타그램 캡처. 그룹 트...
2     [텐아시아류예지 기자] 독보적인 비주얼과 연기력으로 사랑받는 배우들이 ‘2022 A...
3     스타쉽엔터테인먼트 제공 [OSEN장우영 기자] MZ 워너비 아이콘 아이브(IVE :...
4     아이브 안유진. /사진안유진 인스타그램 그룹 아이브(IVE) 멤버 안유진이 근황을 ...
5     아이브 장원영 프로필. 스타쉽엔터테인먼트 제공 걸그룹 아이브(IVE : 안유진 가을...
6     그룹 아이브(IVE) 안유진이 청순한 비주얼을 자랑했다. 사진안유진 인스타그램 지난...
7     아이브(IVE : 안유진 가을 레이 장원영 리즈 이서)의 멤버 장원영이 2년 연속으...
8     아이브 장원영ⓒ 뉴스1 (서울뉴스1) 황미현 기자 아이브(IVE : 안유진 가을 레...
9     /사진제공스타쉽엔터테인먼트 MZ 워너비 아이콘 아이브(IVE 안유진 가을 레이 장원...
10    [텐아시아조준원 기자] 그룹 아이브 멤버 안유진이 근황을 전했다. 안유진은 27일 ...
11    록시땅 제공. 그룹 아이브(IVE) 안유진이 대세로서의 행보를 이어간다. 27일 한...
12    아이브 안유진이 꾸안꾸 매려을 선보였다. 사진안유진 SNS 아이브 안유진이 꾸안꾸 ...
13    유퀴즈 이대호가 은퇴 심경을 전했다. 26일 방송된 tvN 예능프로그램 유 퀴즈 온...
Name: contents, dtype: object

In [38]:
# 검사...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))


In [46]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [53]:

sentences = []

for raw_text in df['contents']:
    try:
        docs = nltk.sent_tokenize(clean_text(raw_text))
        #print(docs)
        for txt in docs:
            if txt.find('▶') > -1 or txt.find('@') > -1 or txt.find('ⓒ') > -1: 
                pass
            else:
                txt = txt.strip()
                if len(txt) > 40:
                    #ok = True
                    #for f in filters:
                    #    if len(f.findall(txt)) > 0:
                    #        ok = False
                    #        print('fitered:',txt)
                    #if ok:
                    if any(chr.isdigit() for chr in txt):
                        pass
                    else:
                        sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        print("Unexpected error:", sys.exc_info()[0])                         

In [54]:
len(sentences)

63

In [55]:
sentences[60]

'이대호는 은퇴 투어 당시 감정으로 올스타저으로 시작됐는데 눈물을 좀 흘렸다.'

In [50]:
df1 = pd.DataFrame({'sentence':sentences})

In [51]:
df1

,sentence
0,한편 스타랭킹은 팬들이 직접 자신의 최애 스타에 투표해 순위를 결정하는 랭킹 투표다.
1,최근 네이트판을 비롯한 각종 온라인 커뮤니티에선 지효 동생 박지영의 미모가 누리꾼들...
2,공개된 화보에는 비니와 검은색 구두를 멋스럽게 소화한 박지영의 모습이 담겨있다.
3,누리꾼들은 “와 진짜 이쁘다 “제니랑 닮은 것 같다 “귀엽다 등의 반응을 보이며 감...
4,대한민국 대표 로코퀸 박민영은 올해에도 JTBC 드라마 ‘기상청 사람들 : 사내연애...
...,...
58,생각도 많이 했었고 제가 스무 살 때 (바꿨다)며 타자로 바꾼 것도 전지훈련에서 어...
59,타자 권유했을 때 해보고 싶다고 한 것이 운이 좋게 맞아떨어졌던 것 같다고 말했다.
60,이대호는 은퇴 투어 당시 감정으로 올스타저으로 시작됐는데 눈물을 좀 흘렸다.
61,제가 어릴 때는 많이 울었지만 프로 생활하면서 독하게 살다 보니까 눈물을 잘 안 흘...


In [52]:
df1.to_csv('korean_sentences.csv')